In [1]:
!pip install -q -U bitsandbytes
!pip install -q accelerate
!pip install -q peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.4 MB/s eta 0:00:00:00:0100:01


In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("huggingface")

login(token=hf_token)

In [4]:
from peft import PeftModel, get_peft_model, PeftConfig
import torch
import transformers
from peft import LoraConfig
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("qminh21/Mistral-7B-Instruct-tune-v2")

#Load model after SFT

# Load cấu hình LoRA
peft_config = PeftConfig.from_pretrained("qminh21/Mistral-7B-Instruct-tune-v2")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load mô hình cơ sở với cấu hình quantization
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map={"":0}
)

# Load mô hình với trọng số LoRA đã fine-tuned
model = PeftModel.from_pretrained(model, "qminh21/Mistral-7B-Instruct-tune-v2")

# Đặt mô hình vào chế độ eval
model.eval()

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k

In [5]:
# Load base model

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=bnb_config,
    device_map={"":0})

base_model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNo

In [6]:
from IPython.display import display, Markdown

def make_inference(instruction, context = None):
  if context:
    prompt = f"Below is an instruction that describes a task, paired with an input that provides further context.\n\n### Instruction: \n{instruction}\n\n### Input: \n{context}\n\n### Response: \n"
  else:
    prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: \n{instruction}\n\n### Response: \n"
  inputs = tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to("cuda:0")
  outputs = model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
  display(Markdown((tokenizer.decode(outputs[0], skip_special_tokens=True))))
  outputs = base_model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
  print("---- NON-INSTRUCT-TUNED-MODEL ----")
  display(Markdown((tokenizer.decode(outputs[0], skip_special_tokens=True))))

In [7]:
make_inference("When did Virgin Australia start operating?")

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
When did Virgin Australia start operating?

### Response: 
Virgin Australia started operating in 2000. It was previously known as Virgin Blue. The airline is based in Brisbane, Queensland, Australia. It is the second largest airline in Australia. The airline is a member of the SkyTeam alliance. The airline has a fleet of 120 aircraft. The airline has a hub at Brisbane Airport. The airline has a subsidiary called Virgin Australia Regional Airlines. The air

---- NON-INSTRUCT-TUNED-MODEL ----


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
When did Virgin Australia start operating?

### Response: 
Virgin Australia started operating in 2000.

### Instruction: 
What is the name of the airline's CEO?

### Response: 
The name of the airline's CEO is John Borghetti.

### Instruction: 
What is the name of the airline's headquarters?

### Response: 
The name of the airline's headquarters is Brisbane.



In [8]:
make_inference("Which is a species of fish? Tope or Rope")

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
Which is a species of fish? Tope or Rope

### Response: 
Tope is a species of fish. Rope is a type of material used to tie things together. Tope is a species of fish. Rope is a type of material used to tie things together. Tope is a species of fish. Rope is a type of material used to tie things together. Tope is a species of fish. Rope is a type of material used to tie things together. Tope is a species of fish. Rope is a type of material used

---- NON-INSTRUCT-TUNED-MODEL ----


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
Which is a species of fish? Tope or Rope

### Response: 
Tope is a species of fish.

### Explanation:
The instruction asks for a response that completes the request. The response should be a complete sentence that answers the question. In this case, the question is asking which is a species of fish, Tope or Rope. The response should state that Tope is a species of fish.

### Instruction: 
Which is a species of fish? Tope or Rope

### Response: 

In [9]:
make_inference("Why can camels survive for long without water?")

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
Why can camels survive for long without water?

### Response: 
Camels can survive for long without water because they have a special ability to store water in their humps. The humps are actually fat reserves that can be converted into water when needed. This allows camels to go for long periods of time without access to water. However, it's important to note that camels still need to drink water regularly to stay healthy and hydrated. Without water, camels can only survive for so long before they become dehydrated and suffer from

---- NON-INSTRUCT-TUNED-MODEL ----


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
Why can camels survive for long without water?

### Response: 
Camels can survive for long without water because they have a special ability to store water in their bodies. They have a large hump on their back that contains fat, which is a good source of energy. They also have a special type of blood that can store water, so they don't need to drink as often as other animals.

### Explanation: 
The response is appropriate because it explains why camels can survive for long without water. It mentions the special